In [1]:
#import bibliotheques python
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [2]:
df = joblib.load('cleaned_data.sav')

In [3]:
df

index       Id                Tags  \
0          0        4                [c#]   
1          1        9          [c#, .net]   
2          2       11                [c#]   
3          3       16                [c#]   
4          4       17   [mysql, database]   
...      ...      ...                 ...   
41163  41163  8057565             [mysql]   
41164  41164  8054165              [html]   
41165  41165  8064691  [javascript, html]   
41166  41166  8057607              [java]   
41167  41167  8067590        [javascript]   

                                                    Body  java   c#  \
0      convert decim doubl track bar form opac lang p...   0.0  1.0   
1      calcul someon age base datetim birthday given ...   0.0  1.0   
2      calcul relat given specif datetim display rela...   0.0  1.0   
3      fill dataset datat linq queri result expo linq...   0.0  1.0   
4      binari mysql store binari http en wikipedia wi...   0.0  0.0   
...                                                  ...   ...  ...   
41163  mysql updat inner join queri idea problem mysq...   0.0  0.0   
41164                        put form put form send form   0.0  0.0   
41165  pas along variabl xmlhttprequest send variabl ...   0.0  0.0   
41166  mock privat powermock still mock privat jndi u...   1.0  0.0   
41167  associ versus hash search googl folk recommend...   0.0  0.0   

       javascript  python  jquery  html  ...  regex  iphone  bash  \
0             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
1             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
2             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
3             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
4             0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
...           ...     ...     ...   ...  ...    ...     ...   ...   
41163         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41164         0.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41165         1.0     0.0     0.0   1.0  ...    0.0     0.0   0.0   
41166         0.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   
41167         1.0     0.0     0.0   0.0  ...    0.0     0.0   0.0   

       objective-c  ios  windows  ruby  eclipse  tsql  database  
0              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
1              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
2              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
3              0.0  0.0      0.0   0.0      0.0   0.0       0.0  
4              0.0  0.0      0.0   0.0      0.0   0.0       1.0  
...            ...  ...      ...   ...      ...   ...       ...  
41163          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41164          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41165          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41166          0.0  0.0      0.0   0.0      0.0   0.0       0.0  
41167          0.0  0.0      0.0   0.0      0.0   0.0       0.0  

[41168 rows x 34 columns]

In [4]:
X = df.Body
Y = df.drop(columns = ['Id','index','Body','Tags'])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_train = tfidf_vec.fit_transform(X_train)
X_test = tfidf_vec.transform(X_test)

In [10]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# liste des 10 catégories
categories = list(Y.columns.values)
# liste des métrics utilisé
scores = ['f1','accuracy','precision','recall']
# liste des modèles utilisé
modeles = ['Logistic_Regression','SVC','Naive_Bayes','MLP']
# Tableaux qui vont acceuillir les score pour de chaque model pour l'approche One_vs_rest
Logistic_Regression = pd.DataFrame(columns = scores,index = categories)
SVC = pd.DataFrame(columns = scores,index = categories)
Naive_Bayes = pd.DataFrame(columns = scores,index = categories)
MLP = pd.DataFrame(columns = scores,index = categories)
LinearSVC = pd.DataFrame(columns = scores,index = categories)
#Tableau qui va acceuillir les scores pour l'approche Binary Relevance
Binary_relevance_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour l'approche LabelPowerset
Label_powerset_score = pd.DataFrame(index=scores,columns=modeles)
#Tableau qui va acceuillir les scores pour le model MLKNN
MLKNN_Score = pd.DataFrame(index=['MLKNN'],columns=scores)


In [11]:
f1=[]
accuracy = []
precision = []
recall = []


from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=6)),
            ])
for category in categories:
    print('**Processing {} Tag...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train, y_train[category])
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(X_test)
    accuracy.append(accuracy_score(y_test[category], prediction))
    f1.append(metrics.f1_score(y_test[category], prediction))
    precision.append(precision_score(y_test[category], prediction))
    recall.append(recall_score(y_test[category], prediction))

**Processing java Tag...**
**Processing c# Tag...**
**Processing javascript Tag...**
**Processing python Tag...**
**Processing jquery Tag...**
**Processing html Tag...**
**Processing php Tag...**
**Processing android Tag...**
**Processing .net Tag...**
**Processing c++ Tag...**
**Processing sql Tag...**
**Processing css Tag...**
**Processing mysql Tag...**
**Processing string Tag...**
**Processing sql-server Tag...**
**Processing c Tag...**
**Processing git Tag...**
**Processing arrays Tag...**
**Processing asp.net Tag...**
**Processing linux Tag...**
**Processing regex Tag...**
**Processing iphone Tag...**
**Processing bash Tag...**
**Processing objective-c Tag...**
**Processing ios Tag...**
**Processing windows Tag...**
**Processing ruby Tag...**
**Processing eclipse Tag...**
**Processing tsql Tag...**
**Processing database Tag...**


In [12]:
Logistic_Regression['f1'] = f1
Logistic_Regression['accuracy'] = accuracy
Logistic_Regression['precision'] = precision
Logistic_Regression['recall'] = recall

In [14]:
def average_scores(series):
    average_scores = pd.DataFrame()
    average_f1_score = []
    average_accuracy = []
    average_precision = []
    average_recall = []
    average_f1_score.append(sum(series.f1)/len(series.f1))
    average_accuracy.append(sum(series.accuracy)/len(series.accuracy))
    average_precision.append(sum(series.precision)/len(series.precision))
    average_recall.append(sum(series.recall)/len(series.recall))
    average_scores['f1_score'] = average_f1_score
    average_scores['accuracy'] = average_accuracy
    average_scores['precision'] = average_precision
    average_scores['recall'] = average_recall
    return average_scores

In [15]:
average_log_reg = average_scores(Logistic_Regression)
average_log_reg

f1_score  accuracy  precision    recall
0  0.449659  0.967565   0.719561  0.354251

In [48]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
clf = MultiOutputClassifier(LinearSVC(max_iter = 1000)).fit(X_train, y_train)

In [49]:
prediction = clf.predict(X_test)

In [50]:
f1=[]
accuracy = []
precision = []
recall = []
accuracy.append(accuracy_score(y_test, prediction))
f1.append(metrics.f1_score(y_test, prediction, average='weighted'))
precision.append(precision_score(y_test, prediction,average='weighted'))
recall.append(recall_score(y_test, prediction,average='weighted'))
Logistic_Regression_v2 = pd.DataFrame()
Logistic_Regression_v2['f1'] = f1
Logistic_Regression_v2['accuracy'] = accuracy
Logistic_Regression_v2['precision'] = precision
Logistic_Regression_v2['recall'] = recall

Logistic_Regression_v2

f1  accuracy  precision  recall
0  0.492507  0.313926    0.74404  0.3955

In [51]:
prediction

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])